# Accumulators Demo

### Demo

In [1]:
import findspark
# TODO: your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/home/matthew/spark-2.3.0-bin-hadoop2.7')

In [2]:
import os
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [3]:
# Get or register a Broadcast variable
def getWordBlacklist(sparkContext):
    if ('wordBlacklist' not in globals()):
        globals()['wordBlacklist'] = SparkContext.broadcast(["a", "b", "c"])
    return globals()['wordBlacklist']

In [4]:
# Get or register an Accumulator
def getDroppedWordsCounter(sparkContext):
    if ('droppedWordsCounter' not in globals()):
        globals()['droppedWordsCounter'] = SparkContext.accumulator(0)
    return globals()['droppedWordsCounter']

In [5]:
def createContext(host, port, outputPath):
    # If you do not see this printed, that means the StreamingContext has been loaded
    # from the new checkpoint
    print("Creating new context")
    if os.path.exists(outputPath):
        os.remove(outputPath)
    sc = SparkContext(appName="PythonStreamingRecoverableNetworkWordCount")
    ssc = StreamingContext(sc, 1)

    # Create a socket stream on target ip:port and count the
    # words in input stream of \n delimited text (eg. generated by 'nc')
    lines = ssc.socketTextStream(host, port)
    words = lines.flatMap(lambda line: line.split(" "))
    wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

    def echo(time, rdd):
        # Get or register the blacklist Broadcast
        blacklist = getWordBlacklist(rdd.context)
        # Get or register the droppedWordsCounter Accumulator
        droppedWordsCounter = getDroppedWordsCounter(rdd.context)

        # Use blacklist to drop words and use droppedWordsCounter to count them
        def filterFunc(wordCount):
            if wordCount[0] in blacklist.value:
                droppedWordsCounter.add(wordCount[1])
                False
            else:
                True

        counts = "Counts at time %s %s" % (time, rdd.filter(filterFunc).collect())
        print(counts)
        print("Dropped %d word(s) totally" % droppedWordsCounter.value)
        print("Appending to " + os.path.abspath(outputPath))
        with open(outputPath, 'a') as f:
            f.write(counts + "\n")

    wordCounts.foreachRDD(echo)
    return ssc

In [6]:
host = 'localhost'
port = 8891
# TODO: Change this path to reflect your own directory if you want to run this demo
checkpoint = "checkpoint"
output = "output/accum.txt"

ssc = StreamingContext.getOrCreate(checkpoint, lambda: createContext(host, int(port), output))

Py4JJavaError: An error occurred while calling o2.tryRecoverFromCheckpoint.
: org.apache.spark.SparkException: Failed to read checkpoint from directory checkpoint
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:369)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.

In [7]:
ssc.start()

NameError: name 'ssc' is not defined

In [8]:
ssc.stop()

NameError: name 'ssc' is not defined

matthew@matthew-VirtualBox:~$ jupyter notebook
[I 19:42:49.359 NotebookApp] Writing notebook server cookie secret to /run/user/1000/jupyter/notebook_cookie_secret
[I 19:42:50.353 NotebookApp] Serving notebooks from local directory: /home/matthew
[I 19:42:50.353 NotebookApp] 0 active kernels
[I 19:42:50.353 NotebookApp] The Jupyter Notebook is running at:
[I 19:42:50.354 NotebookApp] http://localhost:8888/?token=7d0b3212c14ce9dfa12e7626bf5c77b70e5efb34f34d0ede
[I 19:42:50.354 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 19:42:50.356 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=7d0b3212c14ce9dfa12e7626bf5c77b70e5efb34f34d0ede
[I 19:42:53.857 NotebookApp] Accepting one-time-token-authenticated connection from 127.0.0.1
[W 19:43:27.865 NotebookApp] 404 GET /nbextensions/widgets/notebook/js/extension.js?v=20180403194249 (127.0.0.1) 20.22ms referer=http://localhost:8888/notebooks/pyspark-streaming/3_advanced/8_Accumulators%20Demo.ipynb
[I 19:43:29.397 NotebookApp] Kernel started: 822246cf-e240-46b9-a540-94564cf0ac3c
[I 19:43:31.455 NotebookApp] Adapting to protocol v5.1 for kernel 822246cf-e240-46b9-a540-94564cf0ac3c
[I 19:45:28.599 NotebookApp] Starting buffering for 822246cf-e240-46b9-a540-94564cf0ac3c:05F645A554494E3380FF0C9B11D3BC53
[I 19:45:29.008 NotebookApp] Kernel restarted: 822246cf-e240-46b9-a540-94564cf0ac3c
[I 19:45:29.923 NotebookApp] Adapting to protocol v5.1 for kernel 822246cf-e240-46b9-a540-94564cf0ac3c
[I 19:45:29.926 NotebookApp] Restoring connection for 822246cf-e240-46b9-a540-94564cf0ac3c:05F645A554494E3380FF0C9B11D3BC53
[I 19:45:29.926 NotebookApp] Replaying 6 buffered messages
[I 19:47:29.435 NotebookApp] Saving file at /pyspark-streaming/3_advanced/8_Accumulators Demo.ipynb
2018-04-03 19:48:36 WARN  Utils:66 - Your hostname, matthew-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2018-04-03 19:48:36 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2018-04-03 19:48:37 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983188000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983186000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983184000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983182000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983180000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983178000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983176000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983174000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983172000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
2018-04-03 19:48:38 WARN  CheckpointReader:87 - Error reading checkpoint from file file:/home/matthew/pyspark-streaming/3_advanced/checkpoint/checkpoint-1519983170000
java.io.InvalidClassException: org.apache.spark.streaming.Checkpoint; local class incompatible: stream classdesc serialVersionUID = -5529346214493608607, local class serialVersionUID = -8642649206206742116
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1880)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1746)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2037)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1568)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:428)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:172)
	at org.apache.spark.streaming.Checkpoint$$anonfun$deserialize$2.apply(Checkpoint.scala:162)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.streaming.Checkpoint$.deserialize(Checkpoint.scala:175)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:356)
	at org.apache.spark.streaming.CheckpointReader$$anonfun$read$2.apply(Checkpoint.scala:352)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.streaming.CheckpointReader$.read(Checkpoint.scala:352)
	at org.apache.spark.streaming.StreamingContextPythonHelper.tryRecoverFromCheckpoint(StreamingContext.scala:871)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)


## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#accumulators-broadcast-variables-and-checkpoints